<a href="https://colab.research.google.com/github/opencoca/vidsum/blob/master/GPT_J_6B_Text_and_code_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPT-J-6B Inference 

This notebook explores how to work with the [GPT-J-6B model](https://github.com/kingoflolz/mesh-transformer-jax/#GPT-J-6B). See the link for more details about the model, including evaluation metrics and credits.

Please note it takes ~10 minutes for this notebook to fully spin up. Sometimes you will runout of ram when attempting to setup the model. If this happens factory restart the runtime and try again. If it contines to happen ...

Start this notebok by clicking `Runtime > Run all` or clicking `⌘/Ctrl+F9`. Use `⌘` on Mac and `Ctrl` on Windows and Linux. 

Keep an eye on this as it spins up. Google likes to make sure you are not a bot and will stop things if you ignore the "I'm not a robot" popup.

## Prepare it to be used

### Install Dependencies

First we download the model and install some dependencies. This step takes at least 5 minutes (possibly longer depending on server load).

!!! **Make sure you are using a TPU runtime!** !!!

In [ ]:
#@title Install Tensorflow
!pip install tensorflow==2.5.0

     |████████████████████████████████| 454.3 MB 13 kB/s 
     |████████████████████████████████| 1.2 MB 62.8 MB/s 
     |████████████████████████████████| 462 kB 72.2 MB/s 
     |████████████████████████████████| 4.0 MB 64.7 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.41.0
    Uninstalling grpcio-1.41.0:
      Successfully uninstalled grpcio-1.41.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.6.0
    Uninstalling tensorflow-2.6.0:
      Successfully uninstalled tensorflow-2.6.0


In [ ]:
#@title Install GPT-J-6B
!apt install zstd

# the "slim" version contain only bf16 weights and no optimizer parameters, which minimizes bandwidth and memory
!time wget -c https://the-eye.eu/public/AI/GPT-J-6B/step_383500_slim.tar.zstd

!time tar -I zstd -xf step_383500_slim.tar.zstd

!git clone https://github.com/kingoflolz/mesh-transformer-jax.git
!pip install -r mesh-transformer-jax/requirements.txt

# jax 0.2.12 is required due to a regression with xmap in 0.2.13
!pip install mesh-transformer-jax/ jax==0.2.12

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  zstd
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 278 kB of archives.
After this operation, 1,141 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 zstd amd64 1.3.3+dfsg-2ubuntu1.2 [278 kB]
Fetched 278 kB in 1s (391 kB/s)
Selecting previously unselected package zstd.
(Reading database ... 155047 files and directories currently installed.)
Preparing to unpack .../zstd_1.3.3+dfsg-2ubuntu1.2_amd64.deb ...
Unpacking zstd (1.3.3+dfsg-2ubuntu1.2) ...
Setting up zstd (1.3.3+dfsg-2ubuntu1.2) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
--2021-10-22 17:53:27--  https://the-eye.eu/public/AI/GPT-J-6B/step_383500_slim.tar.zstd
Resolving the-eye.eu (the-eye.eu)... 162.213.130.244
Connecting to the-eye.eu (the-eye.eu)|162.213.130.244|:443... connected.
HT

Processing ./mesh-transformer-jax
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 590 kB 4.9 MB/s 
  Created wheel for mesh-transformer: filename=mesh_transformer-0.0.0-py3-none-any.whl size=26330 sha256=0116d226cdd46f7cb4f2b7143a9009c39ac174c86956ca10ac078c543822d060
  Stored in directory: /root/.cache/pip/wheels/56/bd/89/b1f6b2f3d6b938d0c5812ee97756a1afd32521bea293543863
  Created wheel for jax: filename=jax-0.2.12-py3-none-any.whl size=682487 sha256=ecd255113a9ec2ca4988a2fea43bb6fc0b4b398111272a2399df1adb739c11d4
  Stored in directory: /root/.cache/pip/wheels/f6/4d/e5/73eec5070b77f25664c67bd793d4eb97f4

## Setup Model

You may have to restart your runtime before this will work. 



Sometimes the next step errors for some reason, just run it again ¯\\\_(ツ)\_/¯

In [ ]:
#@title Import libraries
import time

import jax
from jax.experimental import maps
import numpy as np
import optax
import transformers

from mesh_transformer.checkpoint import read_ckpt
from mesh_transformer.sampling import nucleaus_sample
from mesh_transformer.transformer_shard import CausalTransformer

  Created wheel for tracery: filename=tracery-0.1.1-py3-none-any.whl size=7696 sha256=531ed5220982cb53a64ea212f69c82e667b8f0fa5efea80c471c39019c1b6c18
  Stored in directory: /root/.cache/pip/wheels/14/38/48/da02ec3e9b648c4b52ffbdae69d9a1434e5cf621435f503486
Successfully built tracery


In [ ]:
#@title Create the network
#@markdown ...loading neural weights from the downloaded files. 
#@markdown
#@markdown  > *This can take around 5 minutes.*
total_batch = per_replica_batch * jax.device_count() // cores_per_replica

network = CausalTransformer(params)

network.state = read_ckpt(network.state, "step_383500/", devices.shape[1])

network.state = network.move_xmap(network.state, np.zeros(cores_per_replica))

/usr/local/lib/python3.7/dist-packages/jax/experimental/maps.py:412: UserWarning: xmap is an experimental feature and probably has bugs!
  warn("xmap is an experimental feature and probably has bugs!")


key shape (8, 2)
in shape (1, 2048)
dp 1
mp 8
Total parameters: 6053381344
read from disk/gcs in 34.3879s


# Run Model

Finally, we are ready to infer with the model! The first sample takes around a minute due to compilation, but after that it should only take about 10 seconds per sample.

Feel free to mess with the different sampling parameters (top_p and temp), as well as the length of the generations (gen_len, causes a recompile when initially changed - recompiles will be cached).

You can also change other things like per_replica_batch in the previous cells to change how many generations are done in parallel. A larger batch has higher latency but higher throughput when measured in tokens generated/s. This is useful for doing things like best-of-n cherry picking.

*Tip for best results: Make sure your prompt does not have any trailing spaces, which tend to confuse the model due to the BPE tokenization used during training.*

In [ ]:
# allow text wrapping in generated output: https://stackoverflow.com/a/61401455
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
def infer(context, top_p=0.9, temp=1.0, gen_len=256):
    tokens = tokenizer.encode(context)

    provided_ctx = len(tokens)
    pad_amount = seq - provided_ctx

    padded_tokens = np.pad(tokens, ((pad_amount, 0),)).astype(np.uint32)
    batched_tokens = np.array([padded_tokens] * total_batch)
    length = np.ones(total_batch, dtype=np.uint32) * len(tokens)

    start = time.time()
    output = network.generate(batched_tokens, length, gen_len, {"top_p": np.ones(total_batch) * top_p, "temp": np.ones(total_batch) * temp})

    samples = []
    decoded_tokens = output[1][0]

    for o in decoded_tokens[:, :, 0]:
      samples.append(f"\033[1m{context}\033[0m{tokenizer.decode(o)}")

    #print(f"completion done in {time.time() - start:06}s")
    #this should instead be added to the object returned infered.samples and infered.completion_time
    return samples




In [ ]:
#@markdown The first time that infrences are run for a given length takes imensly longer than on repeated runs. For this reason we spin up the inference engine with a few sample runs. With this out of the way we can quickly experiment with our infrence engine. 

context = """# A NEW WAY TO WORK IN THE OLD PORT OF MONTREAL AT SPACES CITÉ MULTIMÉDIA

## The Old Port of Montreal is a"""

for i in [32,64,128,256,512,1024]:
  print("- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -")
  print(f" ## Infering context with a gen_len of {i} this should take ~ {i/16} sec. when run again ##")
  print("- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -")
  print(infer(context, gen_len=i)[0])

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
 ## Infering context with a gen_len of 32 this should take ~ 2.0 sec. when run again ##
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# A NEW WAY TO WORK IN THE OLD PORT OF MONTREAL AT SPACES CITÉ MULTIMÉDIA

## The Old Port of Montreal is a vibrant complex of spaces. By nurturing the areas where activities and goods intersect, and by developing them as "places," we create a positive impact.  

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
 ## Infering context with a gen_len of 64 this should take ~ 4.0 sec. when run again ##
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# A NEW WAY TO WORK IN THE OLD PORT OF MONTREAL AT SPACES CITÉ MULTIMÉDIA

## The Old Port of Montreal is a Capital City — and its future is unknown

In our continuing quest to discover the secrets of

# Example prompts

We've compiled the model. At this point it should only take about 15 seconds per sample. The first time you run inference on a new dataset takes longer than usual because of compilation to build the graph and write data into disk buffers for fast access by CPUs/GPUs.

The following parameters can be adjusted: top_p, temp, gen_len. When changing the length of generations (gen_len), recompilation is required to maintain accuracy and consistency in results.

You can also change other things like the number of generations done in parallel and how many samples are generated. In addition, you can increase or decrease the batch size for latency/throughput tradeoffs as well. This is useful when performing best-of-n cherry picking tasks where throughput needs to be maximized with minimum batch time delay on each sample generation process.

To get the best results, make sure your prompt does not have any trailing spaces. This is because BPE tokenization can be confused by them.

## Simple Inference 

### Set Sampling Parameters


In [ ]:
#@markdown ### Leave this alone unless you know what you are doing ... or are just having fun experimenting.
#@markdown 
#@markdown - - - 
#@markdown Top_p controls diversity/quality. The higher our top_p the more rich and nuanced oir inferences can be.
top_p = 0 #@param {type:"slider", min:0, max:1, step:0.1}
#@markdown - - - 
#@markdown Temp controls randomnes. Low temps make for mechanical inferences.
temp = 0.6 #@param {type:"slider", min:0, max:1, step:0.1}
#@markdown - - - 
#@markdown Gen_len adjects the length of text infered.
#@markdown Specifcily it adustes the nember of tokens (~4 per engish word).
#@markdown
#@markdown *Note: Changing gen_len will cause long wait times when first setting lengs that have not been set before.*
#@markdown  
gen_len = 512 #@param {type: "slider", min:64, max: 1024, step: 64}
#@markdown - - - 
#@markdown Generate controls the number of inferences that will be generated.
generate = 1 #@param {type: "slider", min:1, max: 12, step: 1}


### Text examples

In [ ]:
#@title Raw Context 
top_p = 0.9 #@param {type:"slider", min:0, max:1, step:0.1}
temp = 0.9 #@param {type:"slider", min:0, max:1, step:0.1}
gen_len = 256 #@param {type: "slider", min:64, max: 512, step: 64}


#print("Enter/Paste your content. Ctrl-D or Ctrl-Z ( windows ) to save it.")
#context = []
#while True:
#    try:
#        line = input()
#    except EOFError:
#        break
#    context.append(line)

context = """"

Title: Bitcoin back above $50,000 as recovery continues
Date: 23 August
Article:

The price of Bitcoin has risen above $50,000 
(£36,480) for the first time in three months
as the cryptocurrency continues to recover 
from a deep slump.

The coin fell sharply in May after a crackdown
in China and a decision by Elon Musk's Tesla 
not to accept it as payment any more.

But investor sentiment is improving as more 
mainstream financial companies begin using
the digital currency.

Summerized:"""

print(infer(top_p=top_p, temp=temp, gen_len=gen_len, context=context)[0])

"

Title: Bitcoin back above $50,000 as recovery continues
Date: 23 August
Article:

The price of Bitcoin has risen above $50,000 
(£36,480) for the first time in three months
as the cryptocurrency continues to recover 
from a deep slump.

The coin fell sharply in May after a crackdown
in China and a decision by Elon Musk's Tesla 
not to accept it as payment any more.

But investor sentiment is improving as more 
mainstream financial companies begin using
the digital currency.

Summerized: The digital currency saw a massive decline after taking a $24 billion hit in May when regulators in China cracked down on the use of

Ex-chief financial officer at Tesla, Deepak Ahuja, has sold $30,000 worth of BTC following the decision to prohibit the use of the

digital currency at the electric car manufacturer.



The Bitcoin price surged on Monday after a weaker U.S. dollar sent the digital currency 
closer to its highest level in years.

China's currency regulator said on Monday that it will st



INT. COMPUTER LAB – MORNING

A normal computer lab, an office or university, Toronto Canada.
It's about 9:00 in the morning. While the place isn't jammed,
there's a healthy number of people working, drinking coffee, 
reading at their computers.

Two of these people are a YOUNG MAN and a YOUNG WOMAN. The
Young Man has a slight working-class English accent.

             YOUNG MAN
  I want to tell you about what we've been studying.

             YOUNG WOMAN
  Oh? What have you been focusing on? 

             YOUNG MAN
  Feline biology is amazing. For example, we recently learned that the average cat has almost twice as many brain cells as a human. 

             YOUNG WOMAN
  Wow! That's impressive. 

             YOUNG MAN
  You're not impressed? 

             YOUNG WOMAN
  I'm impressed, but I also think I'd like to learn more about how you people live, so I can understand the world a bit better. What do you do for fun? 

             YOUNG MAN
  Ha, Ha! We do lots of things. We l

In [ ]:
#@title Make a title sound more academic 
#@markdown  
improve_this = "we're going ti make an instrument using the potentiometer " #@param {type:"string"}
#@markdown - - -
top_p = 0.9 #@param {type:"slider", min:0, max:1, step:0.1}
temp = 0.9 #@param {type:"slider", min:0, max:1, step:0.1}
gen_len = 64 #@param {type: "slider", min:64, max: 2048, step: 64}
generate = 12 #@param {type: "slider", min:1, max: 12, step: 1}

context = """
Instead of saying, "{improve_this}" a better shibboleth would be, \""""

#Indie Hackers want to click on titles such as 
#use something with more punch

for x in range(generate):
  try:
    print(infer(top_p=top_p, 
                temp=temp, 
                gen_len=gen_len, 
                context=context.format(improve_this = improve_this))[0].split('"')[3])
  except:
    pass

we are going to make a device using a potentiometer
we're going to make a musical instrument using the potentiometer.
we're going to make a case
 we're going to use a potentiometer to make an instrument
We're going ti make a self-timer that can take up to 20 pictures per second.
we're going to make a guitar, with a pot on it.
we're going to make a capacitive oscillator
we're going to make an instrument using a potentiometer
we're going to make an instrument that lets us understand the inside of a pot.
we're going ti make an instrument using the POT-i-totem.
we're going to make a circuit using a potentiometer
we are going to build an instrument using the potentiometer.


In [ ]:
#@title Give headlines some more Clickbait 
improve_this = "Make an instrument using the a light sensor " #@param {type:"string"}
#@markdown - - - - - - - - - - - - - - - - - - - - - - - - 
top_p = 0.9 #@param {type:"slider", min:0, max:1, step:0.1}
temp = 1 #@param {type:"slider", min:0, max:1, step:0.1}
gen_len = 64 #@param {type: "slider", min:64, max: 2048, step: 64}
generate = 12 #@param {type: "slider", min:1, max: 12, step: 1}

context = """
You need to make your title more clickbate. So instead of,
"{improve_this}" use something with more punch such as,\""""


for x in range(generate):
  try:
    print(infer(top_p=top_p, 
                temp=temp, 
                gen_len=gen_len, 
                context=context.format(improve_this = improve_this))[0].split('"')[3])
                #context=context.format(improve_this = improve_this))[0])

  except:
    pass

 Turn lights off when your out
 
 


 
 the Gameboy needs to turn on your TV
 
 
 Make an instrument using the light sensor
Something more clickbate. But as for the first image you gave it is a good start. To help you figure out how to do this I've given you a tutorial on how to make a good app intro and a better description of your app. That is in the read
 Make an instrument that lightens the room using your own living room lights. This will entice your reader to want to click through to read your article.

I don't mean to sound unhelpful or mean-spirited, but the truth is
that we screenwriters are just not that smart. We


See your lights are still on? check out the source code at link below and use your magic lights detector software. Also take a look at the App Inventor site ( is the link below and lots of info about it. You might even want to make a quick tutorial about it for the site here:
Make an instrument to detect electric potential in the air


In [ ]:
#@title Generating inference prompts using tracery
import tracery
from tracery.modifiers import base_english

rules = {
    
    'origin': 'Our team builds #imagine# new online #location# beyond',
    'imagine': ['amazing', 'spectaculare' , 'bold', ],
    'location': ['worlds', 'spaces', 'colabortion']
}

grammar = tracery.Grammar(rules)
grammar.add_modifiers(base_english)

for x in range(generate):
  try:
    print(infer(top_p=top_p, 
                temp=temp, 
                gen_len=gen_len, 
                context=grammar.flatten("#origin#"))[0] + "\n")
  except:
    pass

Our team builds amazing new online worlds beyond what video games alone can achieve.

Your ex-girlfriend is starting to flirt online. Your ex-girlfriend is starting to flirt online. You can find your ex-girlfriend’s email or phone number, but can you find out what she’s up to? With a little detective work, you

Our team builds spectaculare new online colabortion beyond our window funreto and holistic. Your meeting is less than the permissiveness of your given meal is wide to identify. I am my extracorporeal time away and lack the time from doing my drugs my subsequent shelf is doing for a bad texto, because I have found one of the largest artists that

Our team builds spectaculare new online spaces beyond the most popular verticals including Vertical Golf, Lawn Care Services, Zen Lawn Care, Lawn Care, News, Easy Lawn Care, Jobs, Fun Life Events, Local Local Events and much more.

This team is all about verticals that are both intriguing to people and therefore effective in generating r

In [ ]:
#@title Generating inference prompts using tracery
import tracery
from tracery.modifiers import base_english

rules = {
    
    'origin': '#this_semester# Jane #does# #good_bad# in class. We hope that she',
    'this_semester': ['Our focus this semester was learning to program microcomputers, using sensors, and building electronic circuits.','This semester we focused on learning to build electric circuits, reading information from sensors, and programming microcontrollers.'],
    'does': ['works', 'does' , 'particpates', 'performs' ],
    'good_bad': ['well', 'competently', 'baddly']
}

grammar = tracery.Grammar(rules)
grammar.add_modifiers(base_english)

for x in range(generate):
  try:
    print(infer(top_p=top_p, 
                temp=temp, 
                gen_len=gen_len, 
                context=grammar.flatten("#origin#"))[0] + "\n")
  except:
    pass

This semester we focused on learning to build electric circuits, reading information from sensors, and programming microcontrollers. Jane works well in class. We hope that she is able to continue to learn even more electronics through your program. We are looking forward to continue to work with the IEEE group.

]]>3Wicked Electronicshttp://wickedelectronics.co.uk/
Mon, 13 Feb 2013 12:48:45 +0000http://wickedelectronics

This semester we focused on learning to build electric circuits, reading information from sensors, and programming microcontrollers. Jane works well in class. We hope that she has the skills necessary for future endeavors in her life and wish her luck and best of luck in her future endeavors. We can’t wait to see her take on this project and create a unique way to interface with the earth.

We got to know these makers and robotics enthusiast from Vassar as a

This semester we focused on learning to build electric circuits, reading information from sensors, and programm


# Being your own Boss
### Published:
### By:Jim Rohn
##### Tags: goals, drive, gtd
### Site: 

## Table of Content
- **1. Being your own boss**
    - **Introduction**
- **2. Starting your own business**
    - **Introduction**
- **3. The love of ownership**
- **4. Being your own boss, Part 2: Mentors, sponsors, and

# Being your own Boss
### Published:
### By:Jim Rohn
##### Tags: goals, drive, gtd
### Site: 

## Table of Content
* [1  ] Introduction
* [2  ] Learning to be your own Boss
  * [2.1  ] Be the Boss, the Boss Knows It
  * [2.2  ] Learn to manage your manager
  * [2.3  ] Learn to manage yourself


# Being your own Boss
### Published:
### By:Jim Rohn
##### Tags: goals, drive, gtd
### Site: 

## Table of Content

<h1>
## <a id="SectionTitle" href="#">Title</a>
</h1>
<h2>
## <a id="TableTitle" href="#">Title</a>
</h2>

<p>
## <a id="Footnote

# Being your own Boss
### Published:
### By:Jim Rohn
##### Tags: goals, drive, gtd
### Site: 

## Table of Content
###
### Chapter 1 - How 


# COVID-19: What do vaccine passports in Quebec & Ontario mean for Canada's largest provinces? 
### Published:Sep 7, 2021
### By:
##### Tags: COVID-19, Quebec, Ontario, Canada
### Site: https://www.canada.ca/en/public-health/services/diseases/

The passports are certificates that confirm vaccinations and allow people to enjoy eating in restaurants without giving their contact information.
"They are mandatory in Canada, and it's a risk with COVID-19," says David Worsley, senior vice-president of public policy at the Canadian Vaping Association.
"We know there's a risk of being infected, and they're a barrier

# COVID-19: What do vaccine passports in Quebec & Ontario mean for Canada's largest provinces? 
### Published:Sep 7, 2021
### By:
##### Tags: COVID-19, Quebec, Ontario, Canada
### Site: https://www.canada.ca/en/public-health/services/diseases/

The passports are certificates that confirm vaccinations and allow people to enjoy eating at establishments like restaurants and grocery s

## Code Generation

### Set Sampling Parameters


In [ ]:
#@markdown ### Leave this alone unless you know what you are doing ... or are just having fun experimenting.
#@markdown 
#@markdown - - - 
#@markdown Top_p controls diversity/quality. The higher our top_p the more rich and nuanced oir inferences can be.
top_p = 0.9 #@param {type:"slider", min:0, max:1, step:0.1}
#@markdown - - - 
#@markdown Temp controls randomnes. Low temps make for mechanical inferences.
temp = 0.9 #@param {type:"slider", min:0, max:1, step:0.1}
#@markdown - - - 
#@markdown Gen_len adjects the length of text infered.
#@markdown Specifcily it adustes the nember of tokens (~4 per engish word).
#@markdown
#@markdown *Note: Changing gen_len will cause long wait times when first setting lengs that have not been set before.*
#@markdown  
gen_len = 512 #@param {type: "slider", min:64, max: 1024, step: 64}
#@markdown - - - 
#@markdown Generate controls the number of inferences that will be generated.
generate = 1 #@param {type: "slider", min:1, max: 12, step: 1}


### Python Code

# check if a number is prime or not

def is_prime(n):
    if n == 1:
        return True
    if n == 2:
        return False
    if n % 2 == 0:
        return False
    for x in range(3, int(n ** 0.5) + 1, 2):
        if n % x == 0:
            return False
    return True

# main program

# These are just some sample data

nums = [2, 4, 6, 8, 10, 12, 16, 18, 20, 30, 35, 40, 50, 60, 70, 80, 90, 100, 200, 300, 400, 500, 700, 900, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 20000, 300000, 500000, 1000000]

# main loop to keep checking until it finds a prime number

for i in range(2, int(nums.__len__() ** 0.5) + 1, 2):
    if is_prime(i):
        print(i, "is prime")

I understand that the issue is with this function 
# check if a number is prime or not

# these are just some sample data

nums = [2, 4, 6, 8, 10, 12, 16, 18, 20, 30, 35, 40, 50, 60, 70, 80, 90, 100, 200, 300, 400, 500, 700, 900, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 20000, 300000, 500

#!/usr/bin/env python3

# detect security flaws 

person_real_name = "Isabelle Plante"

def online_account_scan(n):
  """This function checks if the online account information is correct 
     and if the password, should be changed.
     It also try to download the configuration file and check if the configuration is correct
     and display the current connection status.
     It is called at the end of a login (in case the online account is changed or an error occurs)"""
  
  print("Scaning the online account...")
  account = n
  print("Current account:", account)
  
  name = "Online Account"
  status = "online"
  data_length = 80
  login_status = n  # login status: online or loggged out.
  connection_status = ""
  command = "name"
  command_status = ""
  
  # Display the connection status
  print(name + " Status:", status, "Connection Status:", connection_status)
  
  # Command to check if the account is the same as the last one or an error occured
  # The login of a failed login can

In [ ]:
#@title Vue Code - Simple { form-width: "500px" }
comment = "guest speakers" #@param {type:"string"}
#variable_n_val = "youtube_video_url" #@param {type:"string"}
name = "tab_1" #@param {type:"string"}
#@markdown - - - 
#@markdown Leave this alone unless you know ...
top_p = 0.8 #@param {type:"slider", min:0, max:1, step:0.1}
temp = 0.9 #@param {type:"slider", min:0, max:1, step:0.1}
gen_len = 512 #@param {type:"slider", min:64, max:1024, step:64}
generate = 1 #@param {type: "slider", min:1, max: 12, step: 1}

context = """
<!-- Vue template for {comment} page -->
<section>
  <div id="app" class="container">
    <div class="content">
      <div id="{name} " class=""".format(comment = comment, 
           name = name)
      
print(infer(top_p=top_p, temp=temp, gen_len=gen_len, context=context)[0])


<!-- Vue template for guest speakers page -->
<section>
  <div id="app" class="container">
    <div class="content">
      <div id="tab_1 " class= "tab">
        <h3>
          <span>Speakers</span>
        </h3>
        <ul class="sidenav">
          <li><a href="{{ link_to('/speakers', '#tab_1') }}">Home</a></li>
          <li><a href="{{ link_to('/speakers/new', '#tab_1') }}">Create New</a></li>
          <li><a href="{{ link_to('/speakers/about', '#tab_1') }}">About</a></li>
        </ul>
        <div class="tab-content">
          <div class="tab-pane fade" id="tab_1">
            <div class="panel-body">
              <div class="row">
                <div class="col-md-12">
                  <div class="text-center">
                    <h2>
                      <span>{{ @speakers.size }}</span>
                      <span class="badge">speakers</span>
                    </h2>
                  </div>
                </div>
              </div>
         


# Examples of GPT genereated code.

The folloming 

Please paste interesting examples below

In [ ]:
# check if a number is prime or not

def is_prime(n):
  """ 
  > returns true if n is a prime,
  > false otherwise.
  """
  for i in range(2, n):
    if n % i == 0:
      return False
  return True

print(is_prime(7))

True


In [ ]:
# Program to check if a number is prime or not

num = 8

# Program to check if a number is prime or not

def is_prime(n):
    while n > 1:
        while n % 2 == 0:
            return False
        n = n / 2
    return True

print(is_prime(num))

False


In [ ]:
# check if a number is prime or not
def is_prime(n):
    for i in range(2, n):
        if n % i == 0:
            return False
    return True

is_prime(8)

False